<a href="https://colab.research.google.com/github/True-Duku/Kakaoku/blob/main/Analyse_Comparee_De_Modeles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importer les modules

import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier

In [37]:
# afficher le fichier
df = pd.read_csv('ENB2012_data.csv',index_col=1)

# changement du format d’affichage par défaut des nombres à virgule flottante en utilisant deux chiffres après la virgule décimale
# pour éviter des corrélation à chiffre entiers
pd.options.display.float_format = '{:.2f}'.format

print(df.info(),df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 768 entries, 514.5 to 808.5
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X3      768 non-null    float64
 2   X4      768 non-null    float64
 3   X5      768 non-null    float64
 4   X6      768 non-null    int64  
 5   X7      768 non-null    float64
 6   X8      768 non-null    int64  
 7   Y1      768 non-null    float64
 8   Y2      768 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 60.0 KB
None          X1     X3     X4   X5  X6   X7  X8    Y1    Y2
X2                                                     
514.50 0.98 294.00 110.25 7.00   2 0.00   0 15.55 21.33
514.50 0.98 294.00 110.25 7.00   3 0.00   0 15.55 21.33
514.50 0.98 294.00 110.25 7.00   4 0.00   0 15.55 21.33
514.50 0.98 294.00 110.25 7.00   5 0.00   0 15.55 21.33
563.50 0.90 318.50 122.50 7.00   2 0.00   0 20.84 28.28


In [38]:
# convertir les colonnes int en float
df['X6'] = df['X6'].astype(float)
df['X8'] = df['X8'].astype(float)


In [39]:
# vérifier le résultat
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 768 entries, 514.5 to 808.5
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X3      768 non-null    float64
 2   X4      768 non-null    float64
 3   X5      768 non-null    float64
 4   X6      768 non-null    float64
 5   X7      768 non-null    float64
 6   X8      768 non-null    float64
 7   Y1      768 non-null    float64
 8   Y2      768 non-null    float64
dtypes: float64(9)
memory usage: 60.0 KB


In [40]:
# Correlations et variables les plus corrélées

u = df.corr()
print('Correlation les plus fortes car les plus proches de 1',u)

Correlation les plus fortes car les plus proches de 1       X1    X3    X4    X5    X6    X7    X8    Y1    Y2
X1  1.00 -0.20 -0.87  0.83  0.00 -0.00 -0.00  0.62  0.63
X3 -0.20  1.00 -0.29  0.28 -0.00 -0.00  0.00  0.46  0.43
X4 -0.87 -0.29  1.00 -0.97 -0.00 -0.00 -0.00 -0.86 -0.86
X5  0.83  0.28 -0.97  1.00  0.00  0.00 -0.00  0.89  0.90
X6  0.00 -0.00 -0.00  0.00  1.00 -0.00 -0.00 -0.00  0.01
X7 -0.00 -0.00 -0.00  0.00 -0.00  1.00  0.21  0.27  0.21
X8 -0.00  0.00 -0.00 -0.00 -0.00  0.21  1.00  0.09  0.05
Y1  0.62  0.46 -0.86  0.89 -0.00  0.27  0.09  1.00  0.98
Y2  0.63  0.43 -0.86  0.90  0.01  0.21  0.05  0.98  1.00


In [41]:
# Nouvelle colonne Total_charges avec Heating_load + cooling_load
# Nouvelle variable charges_classe = deouper en 4 classes selon 3 quantiles
# en  utilisant pd.qcut pour créer la variable charges_classes

df['total_charges'] = df['Y1'] + df['Y2']

charges_classes = pd.qcut(df['total_charges'],q=[0,0.25,0.50,0.75,1],labels=[0,1,2,3])

In [43]:
# Stocker dans une variable data les variables explicatives
# separeer les variables en Apprentissage/Test avec Data comme variable
# explicatives et Charges_classes comme variable cible
# Centrer et reduire les variables explicatives des 2 echantillons

data = df.iloc[:,0:8]

x_train,x_test,y_train,y_test = train_test_split(data,charges_classes,test_size=0.2)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
scaler.fit(x_test)
x_test = scaler.transform(x_test)

In [44]:
# modélisation avec les Plus proches voisins KNN

knn = neighbors.KNeighborsClassifier(n_neighbors = 5)
knn.fit(x_train,y_train)
knn.get_params()

y_pred = knn.predict(x_test)
pd.crosstab(y_test,y_pred,rownames=['classe reelle'],colnames=['classe predite'])
knn.score(x_test,y_test)

0.9155844155844156

In [45]:
# Modélisation avec  Support Vector Machine

clf = svm.SVC(kernel='linear',C=10)
clf.fit(x_train,y_train)
clf.get_params()

y_pred2 = clf.predict(x_test)
pd.crosstab(y_test,y_pred2,rownames=['classe reelle'],colnames=['classe predite'])
clf.score(x_test,y_test)

0.9545454545454546

In [47]:
# Modélisation avec Random Forest

rnd = ensemble.RandomForestClassifier(max_features='sqrt',min_samples_split=2)
rnd.fit(x_train,y_train)
rnd.get_params()

y_pred3 = rnd.predict(x_test)
pd.crosstab(y_test,y_pred3,rownames=['classe reelle'],colnames=['classe predite'])
rnd.score(x_test,y_test)

0.948051948051948

In [48]:
# recherche des paramètres
from sklearn.model_selection import GridSearchCV

In [49]:
# avec KNN train

from sklearn.model_selection import GridSearchCV
parameter = {'n_neighbors':[2,50]}
neig = neighbors.KNeighborsClassifier()
knn = GridSearchCV(neig,parameter)
knn.fit(x_train,y_train)
knn.cv_results_

y_pred = knn.predict(x_test)
print(pd.crosstab(y_test,y_pred,rownames=['classe reelle'],colnames=['classe predite']),'score =',
knn.score(x_test,y_test))

classe predite   0   1   2   3
classe reelle                 
0               33   1   0   0
1                7  34   0   0
2                0   2  41   1
3                0   1   4  30 score = 0.8961038961038961


In [50]:
# Avec Support Vector Machine

parameter2 = {'kernel':('linear','rbf'),'C':[0.1,1,10,50]}
svc = svm.SVC()
clf = GridSearchCV(svc,parameter2)
clf.fit(x_train,y_train)
clf.cv_results_

y_pred2 = clf.predict(x_test)
print(pd.crosstab(y_test,y_pred2,rownames=['classe reelle'],colnames=['classe predite']),'score =',
clf.score(x_test,y_test))

classe predite   0   1   2   3
classe reelle                 
0               34   0   0   0
1                2  39   0   0
2                0   1  40   3
3                0   0   1  34 score = 0.9545454545454546


In [51]:
# Avec Random Forest

parameter3 = {'max_features':('sqrt','log','none'),'min_samples_split':range(2,32,2)}
forest = ensemble.RandomForestClassifier()
rnd = GridSearchCV(forest,parameter3)
rnd.fit(x_train,y_train)
rnd.cv_results_

y_pred3 = rnd.predict(x_test)
print(pd.crosstab(y_test,y_pred3,rownames=['classe reelle'],colnames=['classe predite']),' score = ',
rnd.score(x_test,y_test))

classe predite   0   1   2   3
classe reelle                 
0               34   0   0   0
1                1  40   0   0
2                0   1  40   3
3                0   0   1  34  score =  0.961038961038961


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
150 fits failed out of a total of 225.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils